# Introduction to Torch Spatio-Temporal (TSL)
official documentation: https://torch-spatiotemporal.readthedocs.io/en/latest/index.html

purpose of this notebook:
- data structures
- datasets
- models

In [1]:
import tsl
import torch
import torch_geometric

from torch.optim import Adam
from tsl.datasets import PeMS04, PeMS07, PeMS08, PemsBay
from tsl.datasets import MetrLA

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
print(f"tsl version  : {tsl.__version__}")
print(f"torch version: {torch.__version__}")
print(f"torch_geometric version: {torch_geometric.__version__}")

tsl version  : 0.9.5
torch version: 2.4.0+cu121
torch_geometric version: 2.6.1


### Preprocessing

In [4]:
dataset_PeMS04 = PeMS04(root='data/PeMS04')

download: 0.00B [00:00, ?B/s]


HTTPError: HTTP Error 503: Service Unavailable

In [3]:
dataset_MetrLA = MetrLA(root='data/MetrLA')

print(f"Sampling period: {dataset_MetrLA.freq}")
print(f"Has missing values: {dataset_MetrLA.has_mask}")
print(f"Percentage of missing values: {(1 - dataset_MetrLA.mask.mean()) * 100:.2f}%")
print(f"Has exogenous variables: {dataset_MetrLA.has_covariates}")
print(f"Covariates: {', '.join(dataset_MetrLA.covariates.keys())}")

dataset_MetrLA  # type tsl.datasets.Dataset

Sampling period: <5 * Minutes>
Has missing values: True
Percentage of missing values: 8.11%
Has exogenous variables: True
Covariates: dist


/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/tsl/datasets/metr_la.py:98: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(df.index[0], df.index[-1], freq='5T')
/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/tsl/datasets/metr_la.py:109: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  df = df.replace(to_replace=0., method='ffill')


MetrLA(length=34272, n_nodes=207, n_channels=1)

In [4]:
dataset_pemsbay = PemsBay(root='data/PemsBay')

print(f"Sampling period: {dataset_pemsbay.freq}")
print(f"Has missing values: {dataset_pemsbay.has_mask}")
print(f"Percentage of missing values: {(1 - dataset_pemsbay.mask.mean()) * 100:.2f}%")
print(f"Has exogenous variables: {dataset_pemsbay.has_covariates}")
print(f"Covariates: {', '.join(dataset_pemsbay.covariates.keys())}")

dataset_pemsbay

Sampling period: <5 * Minutes>
Has missing values: True
Percentage of missing values: 0.03%
Has exogenous variables: True
Covariates: dist


/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/tsl/datasets/pems_bay.py:84: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(df.index[0], df.index[-1], freq='5T')
/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/tsl/datasets/pems_bay.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=0, inplace=True)


PemsBay(length=52128, n_nodes=325, n_channels=1)

In [4]:
print(dataset_MetrLA.dist)

[[    0.      inf     inf ...     inf  8114.8 10009.7]
 [    inf     0.   2504.6 ...     inf     inf     inf]
 [    inf  1489.3     0.  ...     inf     inf  9837. ]
 ...
 [    inf     inf     inf ...     0.      inf     inf]
 [ 9599.8     inf     inf ...     inf     0.      inf]
 [10119.9  9374.8     inf ...     inf  9018.7     0. ]]


In [6]:
print(f"Default similarity: {dataset_MetrLA.similarity_score}")
print(f"Available similarity options: {dataset_MetrLA.similarity_options}")
print("==========================================")

sim = dataset_MetrLA.get_similarity("distance")  # or dataset_MetrLA.compute_similarity()

sim.shape

Default similarity: distance
Available similarity options: {'distance'}


(207, 207)

In [5]:
# get_connectivity uses get_similarity under the hood
connectivity = dataset_MetrLA.get_connectivity(threshold=0.1, include_self=False, normalize_axis=1, layout="edge_index")

connectivity[0].shape, connectivity[1].shape

((2, 1515), (1515,))

### Building a torch-ready dataset

convert dataset_MetrLA in (tsl.datasets.Dataset) format to (tsl.data.SpatioTemporalDataset) format

In [6]:
from tsl.data import SpatioTemporalDataset

In [7]:
df = dataset_MetrLA.dataframe()

df.head()

nodes,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
channels,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125,67.125,59.625000,62.750000,...,45.625000,65.500,64.500000,66.428574,66.875,59.375000,69.000000,59.250000,69.000000,61.875
2012-03-01 00:05:00,62.666668,68.555557,65.444443,62.444443,64.444443,68.111115,65.000,65.000,57.444443,63.333332,...,50.666668,69.875,66.666664,58.555557,62.000,61.111111,64.444443,55.888889,68.444443,62.875
2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,65.375000,...,44.125000,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857140,62.000
2012-03-01 00:15:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,65.375000,...,44.125000,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857140,62.000
2012-03-01 00:20:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,65.375000,...,44.125000,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857140,62.000


In [8]:
# subclass of torch.utils.data.Dataset
torch_dataset = SpatioTemporalDataset(
    target=dataset_MetrLA.dataframe(),
    connectivity=connectivity,
    mask=dataset_MetrLA.mask,
    horizon=6,
    window=18,
    stride=1
)

torch_dataset

SpatioTemporalDataset(n_samples=34249, n_nodes=207, n_channels=1)

In [9]:
sample = torch_dataset[0]

"""
    sample is of 
    - type torch_geometric.data.Data
    - shape (window, num_nodes, num_features)
"""
sample.x.shape, sample

(torch.Size([18, 207, 1]),
 Data(
   input=(x=[t=18, n=207, f=1], edge_index=[2, e=1515], edge_weight=[e=1515]),
   target=(y=[t=6, n=207, f=1]),
   has_mask=True
 ))

In [12]:
"""
    t = time steps dimension
    n = node dimension
    e = edge dimension
    f = feature dimension
    b = batch dimension
"""
sample.pattern

{'x': 't n f',
 'mask': 't n f',
 'edge_index': '2 e',
 'edge_weight': 'e',
 'y': 't n f'}

In [13]:
batch = torch_dataset[:5]
batch

StaticBatch(
  input=(x=[b=5, t=18, n=207, f=1], edge_index=[2, e=1515], edge_weight=[e=1515]),
  target=(y=[b=5, t=6, n=207, f=1]),
  has_mask=True
)

In [11]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler

# Normalize data using mean and std computed over time and node dimensions
scalers = {'target': StandardScaler(axis=(0, 1))}

# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

dm.setup()
print(dm)

{Train dataloader: size=24642}
{Validation dataloader: size=2722}
{Test dataloader: size=6849}
{Predict dataloader: None}


In [12]:
# extracting the loaders separately for training, validation and testing (for pytorch-lightning, dm can be plugged in as is)
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

In [13]:
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation


class TimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 rnn_layers: int = 1,
                 gnn_kernel: int = 2):
        super(TimeThenSpaceModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size) # free params learned individually for each node (taming local effects in STGNNs, Cini et al.)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='gru',
                           return_only_last_state=True)
        
        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb)  # temporal processing: x=[b t n f] -> h=[b n f]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n f] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out)
        return x_horizon

In [14]:
hidden_size = 32   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = torch_dataset.n_channels   # 1 channel
n_nodes = torch_dataset.n_nodes         # 207 nodes
horizon = torch_dataset.horizon         # 12 time steps

stgnn = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(stgnn)

TimeThenSpaceModel(
  (encoder): Linear(in_features=1, out_features=32, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=207, embedding_size=32)
  (time_nn): RNN(
    (rnn): GRU(32, 32)
  )
  (space_nn): DiffConv(32, 32)
  (decoder): Linear(in_features=32, out_features=6, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=6)
)


In [24]:
epochs = 1
criterion = nn.MSELoss()
optimizer = Adam(stgnn.parameters(), lr=1e-3)

stgnn.train()

for epoch in tqdm(range(epochs), desc="Epochs"):
    epoch_loss = 0.0
    batch_count = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False)
    
    for batch in progress_bar:
        optimizer.zero_grad()
        
        x, edge_index, edge_weight, y = batch.x, batch.edge_index, batch.edge_weight, batch.y
        
        y_hat = stgnn(x, edge_index, edge_weight)
        loss = criterion(y_hat, y)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        batch_count += 1
        
        progress_bar.set_postfix({"Batch Loss": f"{loss.item():.2f}"})
        break

    avg_epoch_loss = epoch_loss / batch_count
    tqdm.write(f"Epoch {epoch + 1}/{epochs} - Average Loss: {avg_epoch_loss:.2f}")

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

x shape init:  torch.Size([64, 18, 207, 1])
x_emb shape:  torch.Size([64, 18, 207, 32])
h shape (after time):  torch.Size([64, 207, 32])
z shape (after space):  torch.Size([64, 207, 32])
x_out shape:  torch.Size([64, 207, 6])
x_horizon shape:  torch.Size([64, 6, 207, 1])


Epochs: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]

Epoch 1/1 - Average Loss: 3653.49


### Pytorch Lightning

In [27]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

loss_fn = MaskedMAE()

metrics = {'mae': MaskedMAE(),
           'mape': MaskedMAPE(),
           'mae_at_15': MaskedMAE(at=2),  # '2' indicates the third time step,
                                          # which correspond to 15 minutes ahead
           'mae_at_30': MaskedMAE(at=5),
           'mae_at_60': MaskedMAE(at=11)}

# setup predictor
predictor = Predictor(
    model=stgnn,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=loss_fn,               # which loss function to be used
    metrics=metrics                # metrics to be logged during train/val/test
)

In [28]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

trainer = pl.Trainer(max_epochs=1,
                     limit_train_batches=100,  # end an epoch after 100 updates
                     callbacks=[checkpoint_callback])

trainer.fit(predictor, datamodule=dm)

/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /users/hunjael/.conda/envs/torch-st/lib/python3.10/s ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard suppor

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=255` in the `DataLoader` to improve performance.
Only args ['edge_index', 'x', 'edge_weight'] are forwarded to the model (TimeThenSpaceModel).


/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=255` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s, v_num=0, val_mae=6.830, val_mae_at_15=7.440, val_mae_at_30=6.590, val_mae_at_60=0.000, val_mape=0.146, train_mae=216.0, train_mae_at_15=220.0, train_mae_at_30=217.0, train_mae_at_60=0.000, train_mape=3.870]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s, v_num=0, val_mae=6.830, val_mae_at_15=7.440, val_mae_at_30=6.590, val_mae_at_60=0.000, val_mape=0.146, train_mae=216.0, train_mae_at_15=220.0, train_mae_at_30=217.0, train_mae_at_60=0.000, train_mape=3.870]


In [29]:
predictor.load_model(checkpoint_callback.best_model_path)
predictor.freeze()

trainer.test(predictor, datamodule=dm);

/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/tsl/engines/predictor.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  storage = torch.load(filename, l

Testing DataLoader 0: 100%|██████████| 108/108 [00:09<00:00, 11.01it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            7.691833019256592
        test_mae             8.043513298034668
     test_mae_at_15          8.828991889953613
     test_mae_at_30          7.726053237915039
     test_mae_at_60                 0.0
        test_mape           0.17782194912433624
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
